In [1]:
# import ngsolve and webgui
from ngsolve import *
from ngsolve.webgui import Draw

# draw the unit_square shape
Draw(unit_square.shape);

WebGuiWidget(value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3, 'mesh_center': [0.5, 0.5000000000000001, 0…

In [2]:
# import occ geometry tools
from netgen.occ import *

unit_square_wp = Rectangle(1,1) # this actually returns a WorkPlane object
Draw(unit_square_wp.Face());

WebGuiWidget(value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3, 'mesh_center': [0.5, 0.5000000000000001, 0…

In [3]:
geo = OCCGeometry(unit_square.shape, dim = 2) # explicitely state dimension, otherwise dx integrals will vanish
mesh = Mesh(geo.GenerateMesh(maxh = 0.05))
Draw(mesh);

WebGuiWidget(value={'gui_settings': {}, 'ngsolve_version': '6.2.2402-37-g45ed24bde', 'mesh_dim': 2, 'order2d':…

In [4]:
V = H1(mesh)

In [5]:
gfu = GridFunction(V)

print(gfu.vec)

       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
 

In [6]:
scene = Draw (gfu)
from time import sleep
#sleep(5)

for i in range(V.ndof):
    gfu.vec[:] = 0
    gfu.vec[i] = 1
    scene.Redraw()
    #sleep(1)

WebGuiWidget(value={'gui_settings': {}, 'ngsolve_version': '6.2.2402-37-g45ed24bde', 'mesh_dim': 2, 'order2d':…

In [7]:
u,u_ = V.TnT() # get test and trial functions

M = BilinearForm(u*u_*dx).Assemble()
S = BilinearForm(grad(u)*grad(u_)*dx).Assemble()

print(M.mat)

Row 0:   0: 0.000208333   4: 0.000104167   79: 0.000104167
Row 1:   1: 0.000342373   22: 8.54845e-05   23: 8.57023e-05   98: 0.000171187
Row 2:   2: 0.000208333   41: 0.000104167   42: 0.000104167
Row 3:   3: 0.000341422   60: 8.54308e-05   61: 8.52804e-05   135: 0.000170711
Row 4:   0: 0.000104167   4: 0.000763375   5: 0.000126605   79: 0.000255083   80: 0.000277521
Row 5:   4: 0.000126605   5: 0.000739151   6: 0.00010237   80: 0.000267205   81: 0.00024297
Row 6:   5: 0.00010237   6: 0.000593954   7: 9.30173e-05   81: 0.00020396   82: 0.000194607
Row 7:   6: 9.30173e-05   7: 0.000546913   8: 8.93736e-05   82: 0.000184083   83: 0.000180439
Row 8:   7: 8.93736e-05   8: 0.000530878   9: 8.8046e-05   83: 0.000177393   84: 0.000176065
Row 9:   8: 8.8046e-05   9: 0.000525536   10: 8.75104e-05   84: 0.000175257   85: 0.000174722
Row 10:   9: 8.75104e-05   10: 0.000523456   11: 8.73232e-05   85: 0.000174405   86: 0.000174218
Row 11:   10: 8.73232e-05   11: 0.000522872   12: 8.73138e-05   86: 

In [8]:
gfu.Set(exp(-10*(x**2+y**2)))
Draw(gfu);

WebGuiWidget(value={'gui_settings': {}, 'ngsolve_version': '6.2.2402-37-g45ed24bde', 'mesh_dim': 2, 'order2d':…

In [9]:
tau = 0.01
mstarinv = BilinearForm(u*u_*dx+tau**2/4* grad(u)*grad(u_)*dx).Assemble().mat.Inverse()


In [10]:
T = 1
udot = gfu.vec.CreateVector()
udot[:] = 0.

scene = Draw(gfu, deformation=True)
for j in range(int(T/tau)):
    gfu.vec.data += tau/2 * udot
    udot.data -= tau * mstarinv*(S.mat*gfu.vec)
    gfu.vec.data += tau/2 * udot
    scene.Redraw()

WebGuiWidget(value={'gui_settings': {}, 'ngsolve_version': '6.2.2402-37-g45ed24bde', 'mesh_dim': 2, 'order2d':…

In [11]:
print(mesh.GetBoundaries())

('bottom', 'right', 'top', 'left')


In [12]:
# import occ geometry tools
from netgen.occ import *

square = Rectangle(1,1).Face() # this actually returns a WorkPlane object
square.edges.Min(X).name = "left"

In [13]:
f = LinearForm(exp(-10*(y-0.5)**2)*u_*ds("left")).Assemble()

In [14]:
T = 1
udot = gfu.vec.CreateVector()
udot[:] = 0.
gfu.vec[:] = 0.
omega = 15

scene = Draw(gfu, deformation=True)
for j in range(int(T/tau)):
    gfu.vec.data += tau/2 * udot
    udot.data += tau * mstarinv*(1/2*(cos(j*tau*omega)+cos((j+1)*tau*omega))*f.vec-S.mat*gfu.vec)
    gfu.vec.data += tau/2 * udot
    scene.Redraw()

WebGuiWidget(value={'gui_settings': {}, 'ngsolve_version': '6.2.2402-37-g45ed24bde', 'mesh_dim': 2, 'order2d':…